# Design the bound states, sampling backbones for looping while maintaining sequence realism

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties
running on node: dig60


### Set working directory to the root of the crispy_shifty repo
TODO set to projects dir

In [2]:
os.chdir("/home/pleung/projects/crispy_shifty")
# os.chdir("/projects/crispy_shifty")

### One-state design the helix-bound states
TODO

In [3]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "00_design_bound_states"
design_list_file = os.path.join(
    os.getcwd(), "scaffolds/02_make_bound_states/bound_states.list"
)
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")
options = " ".join(
    [
        "out:level 200",
        "corrections::beta_nov16 true",
    ]
)

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.design.one_state_design_bound_state",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="medium",
    memory="5G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    simulation_name=simulation_name,
)

https://docs.anaconda.com/anaconda/install

Run the following command with your desired environment active:
sbatch -a 1-$(cat /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/00_design_bound_states/tasks.cmds | wc -l) /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/00_design_bound_states/run.sh


/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
!sbatch -a 1-$(cat /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/00_design_bound_states/tasks.cmds | wc -l) /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/00_design_bound_states/run.sh

Submitted batch job 3653128


### Collect scorefiles of designed bound states and concatenate
TODO change to projects dir

In [ ]:
sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
from crispy_shifty.utils.io import collect_score_file

simulation_name = "00_design_bound_states"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile
TODO change to projects dir

In [ ]:
sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
scores_df = scores_df.convert_dtypes()

### Setup for plotting

In [ ]:
sns.set(
    context="talk",
    font_scale=1,  # make the font larger; default is pretty small
    style="ticks",  # make the background white with black lines
    palette="colorblind",  # a color palette that is colorblind friendly!
)

### Data exploration

In [ ]:
len(scores_df)  #
scores_df.columns

TODO: remember to use `df.convert_dtypes`

In [ ]:
scores_df["time"] = scores_df.time.astype(float) / 60
scores_df["bb_clash"] = scores_df.bb_clash.astype(float)
scores_df["trimmed_length"] = scores_df.trimmed_length.astype(int)

TODO: check which ones failed

In [ ]:
i = 0
for thing in set(scores_df.parent.values):
    if "_DHR" in thing:
        i += 1
print(i)
print(set(scores_df.parent.values))

In [ ]:
inps = parse_scorefile_linear(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/scores.json")
)
inps = inps.convert_dtypes()

In [ ]:
failed_states = []
outs = {k: 0 for k in set(scores_df.state.values)}
ins = {k: 0 for k in set(inps.state.values)}
for k in sorted(ins.keys()):
    ins[k] = len(inps.loc[inps["state"] == k])
    outs[k] = len(scores_df.loc[scores_df["state"] == k])
    if ins[k] != outs[k]:
        failed_states.append(inps.loc[inps["state"] == k])

    # print(f"ins - outs for {k}: {ins[k] - outs[k]}")

In [ ]:
failed = pd.concat(failed_states)
from crispy_shifty.plotting.utils import histplot_df, pairplot_df

failed["trimmed_length"] = failed.trimmed_length.astype(int)
histplot_df(df=failed, bins=10, cols=["bb_clash", "trimmed_length"])

In [ ]:
pairplot_df(df=failed, cols=["bb_clash", "trimmed_length"])

In [ ]:
for f in list(inps.loc[inps["state"].str.contains("DHR4_p")].index):
    print(f"load {f} ;")
    # print(inps.loc[f].state)

In [ ]:
len(failed)

In [ ]:
from crispy_shifty.plotting.utils import histplot_df, pairplot_df

histplot_df(df=scores_df, bins=10, cols=["time", "trimmed_length"])

In [ ]:
pairplot_df(df=scores_df, cols=["time", "trimmed_length"])

In [ ]:
pairplot_df(df=scores_df, cols=["bb_clash", "time"])

In [ ]:
len(inps[inps["bb_clash"] < 1000])

### TODO Filter the designed states leniently based on shape complimentarity and CMS

### TODO Save a list of outputs

In [ ]:
simulation_name = "00_design_bound_states"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

with open(os.path.join(output_path, "designed_states.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

### Prototyping blocks

In [ ]:
%%time 
import pyrosetta

pyrosetta.init(
#     "-corrections::beta_nov16"
)


sys.path.insert(0, "~/projects/crispy_shifty/") # TODO projects
from crispy_shifty.protocols.design import one_state_design_bound_state

t = one_state_design_bound_state(
        None,
        **{
            'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/scaffolds/02_make_bound_states/decoys/0000/notebooks_02_make_bound_states_ed86bcbab67e41f38322ff2f123d5b1e.pdb.bz2',
        }
)

In [ ]:
for i, tppose in enumerate(t):
    print(tppose.pose.scores)
#     tppose.pose.dump_pdb(f"{i}.pdb")

test crispy shifty array task utils

In [ ]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "test"
design_list_file = os.path.join(
    os.getcwd(), "scaffolds/02_make_bound_states/bound_states.list"
)
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")
options = " ".join(
    [
        "out:level 200",
        # "indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
    ]
)

gen_array_tasks(
    distribute_func="crispy_shifty.utils.io.test_func",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="medium",
    memory="4G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    simulation_name=simulation_name,
)

In [ ]:
!sbatch -a 1-$(cat /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/test/tasks.cmds | wc -l) /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/test/run.sh

In [ ]:
from crispy_shifty.utils.io import test_func, wrapper_for_array_tasks
